# Data Ingestion

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

def get_raw_data():
    # Get file from http://archive.ics.uci.edu/ml/datasets/News+Aggregator
    df = pd.read_csv(
        "./../data/demo_data/NewsAggregatorDataset/newsCorpora.csv", 
        sep='\t', 
        names=['id', 'headline', 'url', 'publisher', 'category', 'story', 'hostname', 'timestamp']
    )
    
    # Category: b = business, t = science and technology, e = entertainment, m = health
    return df[['category', 'headline']]

df = get_raw_data()
df.head()

train_df, test_df = train_test_split(df, test_size=0.2)

x_train = train_df['headline'].values
y_train = train_df['category'].values
x_test = test_df['headline'].values
y_test = test_df['category'].values

# x_train = train_df['headline'].head(100).values
# y_train = train_df['category'].head(100).values
# x_test = test_df['headline'].head(10).values
# y_test = test_df['category'].head(10).values

# Modeling

In [2]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin

from keras.models import Model, Input
from keras.layers import Dense, LSTM, Dropout, Embedding, SpatialDropout1D, Bidirectional, concatenate
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

class KerasTextClassifier:
    __author__ = "Edward Ma"
    __copyright__ = "Copyright 2018, Edward Ma"
    __credits__ = ["Edward Ma"]
    __license__ = "Apache"
    __version__ = "2.0"
    __maintainer__ = "Edward Ma"
    __email__ = "makcedward@gmail.com"
    
    OOV_TOKEN = "UnknownUnknown"
    
    def __init__(self, 
                 max_word_input, word_cnt, word_embedding_dimension, labels, 
                 batch_size, epoch, validation_split,
                 verbose=0):
        self.verbose = verbose
        self.max_word_input = max_word_input
        self.word_cnt = word_cnt
        self.word_embedding_dimension = word_embedding_dimension
        self.labels = labels
        self.batch_size = batch_size
        self.epoch = epoch
        self.validation_split = validation_split
        
        self.label_encoder = None
        self.classes_ = None
        self.tokenizer = None
        
        self.model = self._init_model()
        self._init_label_encoder(y=labels)
        self._init_tokenizer()
        
    def _init_model(self):
        input_layer = Input((self.max_word_input,))
        text_embedding = Embedding(
            input_dim=self.word_cnt+2, output_dim=self.word_embedding_dimension,
            input_length=self.max_word_input, mask_zero=False)(input_layer)
        
        text_embedding = SpatialDropout1D(0.5)(text_embedding)
        
        bilstm = Bidirectional(LSTM(units=256, return_sequences=True, recurrent_dropout=0.5))(text_embedding)
        x = concatenate([GlobalAveragePooling1D()(bilstm), GlobalMaxPooling1D()(bilstm)])
        x = Dropout(0.5)(x)
        x = Dense(128, activation="relu")(x)
        x = Dropout(0.5)(x)
        
        output_layer = Dense(units=len(self.labels), activation="sigmoid")(x)
        model = Model(input_layer, output_layer)
        model.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model
    
    def _init_tokenizer(self):
        self.tokenizer = Tokenizer(
            num_words=self.word_cnt+1, split=' ', oov_token=self.OOV_TOKEN)
    
    def _init_label_encoder(self, y):
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(y)
        self.classes_ = self.label_encoder.classes_
        
    def _encode_label(self, y):
        return self.label_encoder.transform(y)
        
    def _decode_label(self, y):
        return self.label_encoder.inverse_transform(y)
    
    def _get_sequences(self, texts):
        seqs = self.tokenizer.texts_to_sequences(texts)
        return pad_sequences(seqs, maxlen=self.max_word_input, value=0)
    
    def _preprocess(self, texts):
        # Placeholder only.
        return [text for text in texts]
        
    def _encode_feature(self, x):
        self.tokenizer.fit_on_texts(self._preprocess(x))
        self.tokenizer.word_index = {e: i for e,i in self.tokenizer.word_index.items() if i <= self.word_cnt}
        self.tokenizer.word_index[self.tokenizer.oov_token] = self.word_cnt + 1
        return self._get_sequences(self._preprocess(x))
        
    def fit(self, X, y):
        """
            Train the model by providing x as feature, y as label
        
            :params x: List of sentence
            :params y: List of label
        """
        
        encoded_x = self._encode_feature(X)
        encoded_y = self._encode_label(y)
        
        self.model.fit(encoded_x, encoded_y, 
                       batch_size=self.batch_size, epochs=self.epoch, 
                       validation_split=self.validation_split)
        
    def trasnform(self, X):
        return self._get_sequences(self._preprocess(X))
        
    def predict_proba(self, X, y=None):
        encoded_x = self.trasnform(X)
        return self.model.predict(encoded_x)
    
    def predict(self, X, y=None):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def predict_decode(self, X, y=None):
        return self._decode_label(self.predict(X))
    
keras_clf = KerasTextClassifier(
    max_word_input=100, word_cnt=30000, word_embedding_dimension=100, 
    labels=list(set(y_train)), batch_size=1024, epoch=1, validation_split=0.1)

Using TensorFlow backend.
/anaconda/envs/py35/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
keras_clf.fit(x_train, y_train)

Train on 304141 samples, validate on 33794 samples
Epoch 1/1
304141/304141 [==============================] - 1294s 4ms/step - loss: 0.6203 - acc: 0.7347 - val_loss: 0.1901 - val_acc: 0.9354


In [4]:
print('Num of Word: %d' % len(keras_clf.tokenizer.word_index))

Num of Word: 30001


# Model Interpretation

In [5]:
from anchor import anchor_text

In [6]:
import spacy
spacy_nlp = spacy.load('en')

In [7]:
import random

In [8]:
%reload_ext  autoreload
%autoreload 2

In [10]:
# number_of_sample = 1
# sample_ids = [random.randint(0, len(x_test) -1 ) for p in range(0, number_of_sample)]
sample_ids = [2]

for idx in sample_ids:
    print('Index: %d, Feature: %s' % (idx, x_test[idx]))
    print('True Label: %s' % (y_test[idx]))
    labels = keras_clf.classes_.tolist()
    estimator = keras_clf.predict

    explainer = anchor_text.AnchorText(spacy_nlp, labels, use_unk_distribution=True)
    exp = explainer.explain_instance(x_test[idx], estimator, threshold=0.8, use_proba=True, batch_size=30)

    max_pred = 2
    print('Key Singal from Anchors: %s' % (' AND '.join(exp.names())))
    print('Precision: %.2f' % exp.precision())
    print()

    exp.show_in_notebook()

Index: 2, Feature: India to have third-largest Twitter population by 2014
True Label: b
Key Singal from Anchors: India
Precision: 1.00

